In [34]:
class IntDevice():
    def __init__(self, program, i=0):
        self.program = program
        self.i = i
        self.base_offset = 0
        self.outbuf = []
        self.inbuf = []
        
        
    def resolve_arg(self, argmode, j):
        if argmode == 0:
            return self.program[self.i+j]
        elif argmode == 1:
            return self.i+j
        elif argmode == 2:
            return self.base_offset + self.program[self.i+j]


    def run(self):
        while True:
            if self.i >= len(self.program):
                print('end of program')
                return self.outbuf
            
            opcode = self.program[self.i]
#             print(opcode)

            op = opcode % 100
            argmodes = []
            argcode = opcode // 100
            while argcode > 0:
                argmodes.append(argcode % 10)
                argcode = argcode // 10

            # ADD A B -> C
            # --------------------------------------------------------
            if op == 1:
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx1 = self.resolve_arg(argmode, 1)
                
                argmode = 0 if len(argmodes) < 2 else argmodes[1]
                idx2 = self.resolve_arg(argmode, 2)
                
                argmode = 0 if len(argmodes) < 3 else argmodes[2]
                idx3 = self.resolve_arg(argmode, 3)

                bigdex = max([idx1, idx2, idx3])
                if len(self.program) <= bigdex:
                    self.program += [0]*(bigdex - len(self.program) + 1)

                arg1 = self.program[idx1]
                arg2 = self.program[idx2]                

                self.program[idx3] = arg1 + arg2
                self.i += 4
            # MULTIPLY A B ->
            # --------------------------------------------------------
            elif op == 2:
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx1 = self.resolve_arg(argmode, 1)
                
                argmode = 0 if len(argmodes) < 2 else argmodes[1]
                idx2 = self.resolve_arg(argmode, 2)
                
                argmode = 0 if len(argmodes) < 3 else argmodes[2]
                idx3 = self.resolve_arg(argmode, 3)

                bigdex = max([idx1, idx2, idx3])
                if len(self.program) <= bigdex:
                    self.program += [0]*(bigdex - len(self.program) + 1)

                arg1 = self.program[idx1]
                arg2 = self.program[idx2]                

                self.program[idx3] = arg1 * arg2
                self.i += 4
            # INPUT -> A
            # --------------------------------------------------------
            elif op == 3:
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx = self.resolve_arg(argmode, 1)

                if len(self.program) <= idx:
                    self.program += [0]*(idx - len(self.program) + 1)

                if len(self.inbuf) > 0:
                    self.program[idx] = self.inbuf[0]
                    self.inbuf = self.inbuf[1:]
                else:
                    self.program[idx] = -1
                    self.i += 2
                    return []
#                     self.program[idx] = int(input('>>> '))
                self.i += 2
            # OUTPUT A
            # --------------------------------------------------------
            elif op == 4:
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx = self.resolve_arg(argmode, 1)
                self.outbuf.append(self.program[idx])
                
                if len(self.outbuf) == 3:
                    outbuf = self.outbuf.copy()
                    self.outbuf = []
                    self.i += 2
                    return outbuf
#                 if self.program[idx] == ord('\n'):
#                     print(''.join([chr(o) for o in outbuf[:-1]]))
#                     self.outbuf = []
                self.i += 2
            # JUMP TRUE A != 0 -> B
            # --------------------------------------------------------
            elif op == 5:                
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx1 = self.resolve_arg(argmode, 1)

                argmode = 0 if len(argmodes) < 2 else argmodes[1]
                idx2 = self.resolve_arg(argmode, 2)

                bigdex = max([idx1, idx2])
                if len(self.program) <= bigdex:
                    self.program += [0]*(bigdex - len(self.program) + 1)

                arg1 = self.program[idx1]
                arg2 = self.program[idx2]    
                if not arg1 == 0:
                    self.i = arg2
                else:
                    self.i += 3
            # JUMP FALSE A == 0 -> B
            # --------------------------------------------------------
            elif op == 6:
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx = self.resolve_arg(argmode, 1)
                arg1 = self.program[idx]

                argmode = 0 if len(argmodes) < 2 else argmodes[1]
                idx = self.resolve_arg(argmode, 2)
                arg2 = self.program[idx]

                if len(self.program) <= arg2:
                    self.program += [0]*(arg2-len(self.program)+1)

                if arg1 == 0:
                    self.i = arg2
                else:
                    self.i += 3
            # LESS THAN A < B 1 -> C
            # --------------------------------------------------------
            elif op == 7:
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx = self.resolve_arg(argmode, 1)
                arg1 = self.program[idx]

                argmode = 0 if len(argmodes) < 2 else argmodes[1]
                idx = self.resolve_arg(argmode, 2)
                arg2 = self.program[idx]

                argmode = 0 if len(argmodes) < 3 else argmodes[2]
                idx = self.resolve_arg(argmode, 3)

                if len(self.program) <= idx:
                    self.program += [0]*(idx-len(self.program)+1)

                if arg1 < arg2:
                    self.program[idx] = 1
                else:
                    self.program[idx] = 0
                self.i += 4
            # EQUALS A == B 1 -> C
            # --------------------------------------------------------
            elif op == 8:
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx = self.resolve_arg(argmode, 1)
                arg1 = self.program[idx]

                argmode = 0 if len(argmodes) < 2 else argmodes[1]
                idx = self.resolve_arg(argmode, 2)
                arg2 = self.program[idx]

                argmode = 0 if len(argmodes) < 3 else argmodes[2]
                idx = self.resolve_arg(argmode, 3)

                if len(self.program) <= idx:
                    self.program += [0]*(idx-len(self.program)+1)

                if arg1 == arg2:
                    self.program[idx] = 1
                else:
                    self.program[idx] = 0
                self.i += 4
            # SHIFT RELATIVE BASE
            # --------------------------------------------------------
            elif op == 9:
                argmode = 0 if len(argmodes) < 1 else argmodes[0]
                idx = self.resolve_arg(argmode, 1)
                arg1 = self.program[idx]
                self.base_offset += arg1
                if len(self.program) <= self.base_offset:
                    self.program += [0]*(self.base_offset-len(self.program)+1)
                self.i += 2
            # HALT
            # --------------------------------------------------------
            elif op == 99:
                return self.outbuf
            # INVALID
            # --------------------------------------------------------
            else:
                print(self.program[i], 'is not an instruction')
                return self.outbuf

In [35]:
with open('calca_09_input.txt', 'r') as infile:
    program = [[int(col) for col in row.strip().split(',')] for row in infile][0]
    
device = IntDevice(program)
device.inbuf = [1]
outbuf = device.run()
print(outbuf)

[3780860499]


In [36]:
with open('calca_23_input.txt', 'r') as infile:
    program = [[int(col) for col in row.strip().split(',')] for row in infile][0]

N = 50
msgqueue = []
devices = []

# initialisation
for i in range(N):
    device_i = IntDevice(program.copy())
    device_i.inbuf.append(i)
    devices.append(device_i)
    
    
# running
done = False
cycle = 1
while not done:
    for i in range(len(devices)):
#         print('device', i, 'inbuf', devices[i].inbuf)
        outbuf_i = devices[i].run()
        print('cycle', cycle, 'device', i, 'outbuf', outbuf_i)
        if len(outbuf_i) == 3:
            dst = outbuf_i[0]
            x = outbuf_i[1]
            y = outbuf_i[2]
            
            if dst < N:
                devices[dst].inbuf += [x, y]  
            elif dst == 255:
                print('first message to 255', x, y)
                done = True
                break
            else:
                msgqueue.append((dst, x, y))
    cycle += 1
    print()
                
for msg in msgqueue:
    print(msg)

cycle 1 device 0 outbuf []
cycle 1 device 1 outbuf []
cycle 1 device 2 outbuf []
cycle 1 device 3 outbuf []
cycle 1 device 4 outbuf []
cycle 1 device 5 outbuf []
cycle 1 device 6 outbuf []
cycle 1 device 7 outbuf []
cycle 1 device 8 outbuf []
cycle 1 device 9 outbuf []
cycle 1 device 10 outbuf []
cycle 1 device 11 outbuf []
cycle 1 device 12 outbuf []
cycle 1 device 13 outbuf []
cycle 1 device 14 outbuf []
cycle 1 device 15 outbuf []
cycle 1 device 16 outbuf []
cycle 1 device 17 outbuf []
cycle 1 device 18 outbuf []
cycle 1 device 19 outbuf []
cycle 1 device 20 outbuf []
cycle 1 device 21 outbuf []
cycle 1 device 22 outbuf []
cycle 1 device 23 outbuf []
cycle 1 device 24 outbuf []
cycle 1 device 25 outbuf []
cycle 1 device 26 outbuf []
cycle 1 device 27 outbuf []
cycle 1 device 28 outbuf []
cycle 1 device 29 outbuf []
cycle 1 device 30 outbuf []
cycle 1 device 31 outbuf []
cycle 1 device 32 outbuf []
cycle 1 device 33 outbuf []
cycle 1 device 34 outbuf []
cycle 1 device 35 outbuf []
cy

cycle 6 device 41 outbuf []
cycle 6 device 42 outbuf []
cycle 6 device 43 outbuf []
cycle 6 device 44 outbuf []
cycle 6 device 45 outbuf []
cycle 6 device 46 outbuf []
cycle 6 device 47 outbuf []
cycle 6 device 48 outbuf []
cycle 6 device 49 outbuf []

cycle 7 device 0 outbuf [32, 119859, 21322]
cycle 7 device 1 outbuf []
cycle 7 device 2 outbuf []
cycle 7 device 3 outbuf []
cycle 7 device 4 outbuf []
cycle 7 device 5 outbuf [18, 3548, 1027626316992]
cycle 7 device 6 outbuf []
cycle 7 device 7 outbuf []
cycle 7 device 8 outbuf []
cycle 7 device 9 outbuf []
cycle 7 device 10 outbuf [48, 41506, 82189]
cycle 7 device 11 outbuf []
cycle 7 device 12 outbuf []
cycle 7 device 13 outbuf []
cycle 7 device 14 outbuf []
cycle 7 device 15 outbuf []
cycle 7 device 16 outbuf []
cycle 7 device 17 outbuf []
cycle 7 device 18 outbuf []
cycle 7 device 19 outbuf []
cycle 7 device 20 outbuf []
cycle 7 device 21 outbuf []
cycle 7 device 22 outbuf []
cycle 7 device 23 outbuf []
cycle 7 device 24 outbuf []
c

In [37]:
with open('calca_23_input.txt', 'r') as infile:
    program = [[int(col) for col in row.strip().split(',')] for row in infile][0]

N = 50
natmsg = []
prevnat = []
devices = []

# initialisation
for i in range(N):
    device_i = IntDevice(program.copy())
    device_i.inbuf.append(i)
    devices.append(device_i)
    
# running
done = False
cycle = 1
maxcycle = 1000
while not done:
    if cycle == maxcycle:
        print('time out')
        break
    idle = True
    
    for i in range(len(devices)):
        outbuf_i = devices[i].run()
        if len(outbuf_i) == 3:
            idle = False
            print('cycle', cycle, 'device', i, 'outbuf', outbuf_i)
            dst = outbuf_i[0]
            x = outbuf_i[1]
            y = outbuf_i[2]
            
            if dst < N:
                devices[dst].inbuf += [x, y]  
            elif dst == 255:
                natmsg = [dst, x, y]
            else:
                msgqueue.append((dst, x, y))
                
    if idle and len(natmsg) == 3:
        devices[0].inbuf += [natmsg[1], natmsg[2]]
        if natmsg == prevnat:
            print(prevnat, natmsg)
            done = True
        else:
            prevnat = natmsg.copy()
        
    cycle += 1
    print()
                
for msg in msgqueue:
    print(msg)


cycle 2 device 0 outbuf [26, 31718, 21322]
cycle 2 device 4 outbuf [47, 284343, 19]
cycle 2 device 6 outbuf [11, 319892, 1063]
cycle 2 device 7 outbuf [33, 17607, 3]
cycle 2 device 9 outbuf [23, 220953, 11]
cycle 2 device 12 outbuf [23, 147302, 8]
cycle 2 device 14 outbuf [33, 29345, 984]
cycle 2 device 17 outbuf [47, 189562, 334]
cycle 2 device 19 outbuf [3, 3457, 6599]
cycle 2 device 21 outbuf [47, 94781, 18]
cycle 2 device 22 outbuf [24, 2689, 5]
cycle 2 device 27 outbuf [3, 10371, 2]
cycle 2 device 28 outbuf [25, 88612, -186]
cycle 2 device 29 outbuf [2, 797, 1]
cycle 2 device 31 outbuf [24, 8067, 10]
cycle 2 device 34 outbuf [10, 65542, 82189]
cycle 2 device 37 outbuf [25, 44306, 41]
cycle 2 device 38 outbuf [24, 10756, 125]
cycle 2 device 39 outbuf [49, 269391, 160]
cycle 2 device 40 outbuf [33, 5869, 31]
cycle 2 device 41 outbuf [2, 1594, 22709]
cycle 2 device 42 outbuf [23, 294604, -128]

cycle 3 device 0 outbuf [35, 102359, 21322]
cycle 3 device 2 outbuf [20, 170349, 22709]
c

cycle 50 device 44 outbuf [255, 102859, 16307]

cycle 51 device 0 outbuf [32, 119859, 16384]
cycle 51 device 18 outbuf [44, 33073, 16174699590848]
cycle 51 device 32 outbuf [18, 2661, 4398046511104]
cycle 51 device 44 outbuf [255, 102859, 16174]

cycle 52 device 18 outbuf [44, 33073, 16045850571968]
cycle 52 device 44 outbuf [255, 102859, 16045]


cycle 54 device 0 outbuf [26, 31718, 16045]
cycle 54 device 26 outbuf [18, 887, 22428832365040]

cycle 55 device 0 outbuf [35, 102359, 16045]
cycle 55 device 18 outbuf [44, 33073, 15571972468400]
cycle 55 device 35 outbuf [18, 1774, -12028395847680]
cycle 55 device 44 outbuf [255, 102859, 15571]

cycle 56 device 0 outbuf [35, 204718, 16045]
cycle 56 device 18 outbuf [44, 33073, 15826109345456]
cycle 56 device 35 outbuf [18, 1774, -11779517295900]
cycle 56 device 44 outbuf [255, 102859, 15826]

cycle 57 device 0 outbuf [32, 39953, 16045]
cycle 57 device 18 outbuf [44, 33073, 16074987897236]
cycle 57 device 32 outbuf [18, 2661, 4307046891520]
c

cycle 152 device 44 outbuf [255, 102859, 15258]

cycle 153 device 0 outbuf [32, 39953, 15261]
cycle 153 device 18 outbuf [44, 33073, 15261794617244]
cycle 153 device 32 outbuf [18, 2661, 3556587587316]
cycle 153 device 44 outbuf [255, 102859, 15261]

cycle 154 device 0 outbuf [32, 79906, 15261]
cycle 154 device 18 outbuf [44, 33073, 15260629363464]
cycle 154 device 32 outbuf [18, 2661, 3555422715186]
cycle 154 device 44 outbuf [255, 102859, 15260]

cycle 155 device 0 outbuf [32, 119859, 15261]
cycle 155 device 18 outbuf [44, 33073, 15259464491334]
cycle 155 device 32 outbuf [18, 2661, 3554258224581]
cycle 155 device 44 outbuf [255, 102859, 15259]

cycle 156 device 18 outbuf [44, 33073, 15258300000729]
cycle 156 device 44 outbuf [255, 102859, 15258]


cycle 158 device 0 outbuf [26, 31718, 15258]
cycle 158 device 26 outbuf [18, 887, 21328708272096]

cycle 159 device 0 outbuf [35, 102359, 15258]
cycle 159 device 18 outbuf [44, 33073, 15254106389193]
cycle 159 device 35 outbuf [18, 1774, -

In [25]:
dev = IntDevice(program.copy())
dev.inbuf.append(0)

In [28]:
o = dev.run()
print(o)

3
[]
